In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


In [2]:
database="dsc10742_gcctmsd_lz_db"
tab_list=['account','activityparty','annotation','appointment','businessunit','calendar','calendarrule','contact','cxlvhlp_chatactivity','cxlvhlp_chatqueuestatistic','cxlvhlp_surveyitem','email','fax','gcct_accountresponsibleagent','gcct_additionalsymptomcodes','gcct_addportalmessage','gcct_arbitrationclaimprocessing','gcct_buybackevaluationmilestones','gcct_caseassignment','gcct_caseclassification','gcct_casedispositiontype','gcct_coachback','gcct_country','gcct_customersatisfactiontools','gcct_delegationofauthority','gcct_demandltrtpsmclaimsprocessing','gcct_doaprogramcode','gcct_documentcustomerrecontact','gcct_engine','gcct_executive','gcct_executiveliaison','gcct_fieldinvolvementassistancerequest','gcct_fmcccontract','gcct_fulfillmentvendor','gcct_genericinformation','gcct_genericinformationtopic','gcct_generictopic','gcct_goodfaithreview','gcct_knownpartsdelay','gcct_lemonlawcriteria','gcct_loanercost','gcct_loyaltyassistance','gcct_loyaltycostdetails','gcct_loyaltyprogramcode','gcct_material','gcct_materialrequest','gcct_materialrequestdetail','gcct_offer','gcct_offerdetail','gcct_onlinegaragevehicles','gcct_partsorderstatus','gcct_partsordersystems','gcct_partssmeassistancerequest','gcct_pcarequest','gcct_priordealerdecision','gcct_qaincident','gcct_qamonitoring','gcct_queuepriority','gcct_ravprocessingmilestones','gcct_reactdata','gcct_reacttransmission','gcct_reasoncode','gcct_recall','gcct_region','gcct_rentaldetail','gcct_slaconfiguration','gcct_sms','gcct_socialmediaconversation','gcct_socialmediamessage','gcct_socialmediaprofile','gcct_specialloanercode','gcct_state','gcct_surveyconfiguration','gcct_symptomclassification','gcct_tasktype','gcct_technicalassistancerequest','gcct_testdriverequest','gcct_timezone','gcct_userdelegationofauthority','gcct_vehicle','gcct_vehiclebrand','gcct_vehicleclass','gcct_vehicleloyaltyallowance','gcct_vehicleoffroad','gcct_vehicleowner','gcct_warrantyhistory','gcct_warrantyloanerpartsdelay','gcct_warrantyloanerrequest','gcct_warrantyloanerrequesthistory','globaloptionsetmetadata','incident','incidentresolution','letter','msdyn_answer','msdyn_question','msdyn_questionresponse','msdyn_survey','msdyn_surveyinvite','msdyn_surveyresponse','optionsetmetadata','phonecall','queue','queueitem','sla','slaitem','slakpiinstance','socialactivity','socialprofile','statemetadata','statusmetadata','systemuser','task','team','teammembership','territory','gcct_marketingprogram','gcct_catalogcorrection','gcct_cataloginquiry','gcct_dealerclaims','gcct_fcsinquiry','gcct_partavailabilityinquiry','gcct_partorderstatusinquiry','gcct_partqualitycontrolinquiry','gcct_pricinginquiries','gcct_trafficinquiry','gcct_catalogdeck', 'gcct_partdistributioncenter', 'gcct_portalcasetype']

sqlContext.sql("use "+database)
today=datetime.today().strftime('%Y%m%d')

In [2]:
#Writes schema to excel file. Sheet name as table name
writer = pd.ExcelWriter("test3.xlsx", engine = 'xlsxwriter')

for m in sqlContext.tables(database).select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
            k=m.asDict().values()[0].encode('utf-8')
            k2=k[:31]
            g1=sqlCtx.sql('desc '+k).toPandas()
            g1.to_excel(writer, sheet_name=k2, index=False)

writer.save()
writer.close()

# Prod use xlsx
# QA use CSV

# PROD

In [3]:
#Writes schema to excel file. All in one sheet
df= pd.DataFrame(columns = ['col_name','data_type','table'])
file_name="GCCT_schema_DSC_PROD_Sep_19_2019_LZ.xlsx"
writer = pd.ExcelWriter(file_name, engine = 'xlsxwriter')


#for m in sqlContext.tables(database).select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
for k in tab_list:
            #k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('desc '+k).toPandas()
            g1['table']=k
            g1=g1[['col_name','data_type','table']]
            df=df.append(g1, ignore_index=True)
            


df.to_excel(writer, sheet_name="sheet1", index=False)

writer.save()
writer.close()

commands.getoutput("cp "+file_name+" /s/latluri1/GCCT_DES_DSC_monitoring/PROD/archive")

''

# QA

In [3]:
#Writes schema to excel file. All in one sheet CSV 
df= pd.DataFrame(columns = ['col_name','data_type','table'])


#for m in sqlContext.tables(database).select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
for k in tab_list:
            #k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('desc '+k).toPandas()
            g1['table']=k
            g1=g1[['col_name','data_type','table']]
            df=df.append(g1, ignore_index=True)
             


df.to_csv("/s/latluri1/GCCT_DES_DSC_monitoring/PPROD/archive/GCCT_schema_DSC_QA_Aug_19_2019_LZ.csv",index=False)



In [3]:
database="lacek_fpr"
sqlContext.sql("use "+database)
#Writes schema to excel file. All in one sheet CSV 
df= pd.DataFrame(columns = ['col_name','data_type','table'])


for m in sqlContext.tables(database).collect():
#for k in tab_list:
            k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('desc '+k).toPandas()
            g1['table']=k
            g1=g1[['col_name','data_type','table']]
            df=df.append(g1, ignore_index=True)
            


df.to_csv("lacek_fpr_comarch.csv",index=False)



In [22]:
df.table.describe()

count       12341
unique        115
top       contact
freq          431
Name: table, dtype: object

In [4]:
df.to_csv("/s/latluri1/GCCT_DES_DSC_monitoring/PROD/GCCT_schema_DSC_PROD_"+today+".csv",index=False)

In [7]:
#Writes schema to excel file. All in one sheet uses the list of tables
df= pd.DataFrame(columns = ['col_name','data_type','table'])
writer = pd.ExcelWriter("GCCT_schema_Mar19.xlsx", engine = 'xlsxwriter')
tab_list=['account','activityparty','annotation','appointment','businessunit','calendar','calendarrule','contact','cxlvhlp_chatactivity','cxlvhlp_chatqueuestatistic','cxlvhlp_surveyitem','email','fax','gcct_accountresponsibleagent','gcct_additionalsymptomcodes','gcct_addportalmessage','gcct_arbitrationclaimprocessing','gcct_buybackevaluationmilestones','gcct_caseassignment','gcct_caseclassification','gcct_casedispositiontype','gcct_coachback','gcct_country','gcct_customersatisfactiontools','gcct_delegationofauthority','gcct_demandltrtpsmclaimsprocessing','gcct_doaprogramcode','gcct_documentcustomerrecontact','gcct_engine','gcct_executive','gcct_executiveliaison','gcct_fieldinvolvementassistancerequest','gcct_fmcccontract','gcct_fulfillmentvendor','gcct_genericinformation','gcct_genericinformationtopic','gcct_generictopic','gcct_goodfaithreview','gcct_knownpartsdelay','gcct_lemonlawcriteria','gcct_loanercost','gcct_loyaltyassistance','gcct_loyaltycostdetails','gcct_loyaltyprogramcode','gcct_material','gcct_materialrequest','gcct_materialrequestdetail','gcct_offer','gcct_offerdetail','gcct_onlinegaragevehicles','gcct_partsorderstatus','gcct_partsordersystems','gcct_partssmeassistancerequest','gcct_pcarequest','gcct_priordealerdecision','gcct_qaincident','gcct_qamonitoring','gcct_queuepriority','gcct_ravprocessingmilestones','gcct_reactdata','gcct_reacttransmission','gcct_reasoncode','gcct_recall','gcct_region','gcct_rentaldetail','gcct_slaconfiguration','gcct_sms','gcct_socialmediaconversation','gcct_socialmediamessage','gcct_socialmediaprofile','gcct_specialloanercode','gcct_state','gcct_surveyconfiguration','gcct_symptomclassification','gcct_tasktype','gcct_technicalassistancerequest','gcct_testdriverequest','gcct_timezone','gcct_userdelegationofauthority','gcct_vehicle','gcct_vehiclebrand','gcct_vehicleclass','gcct_vehicleloyaltyallowance','gcct_vehicleoffroad','gcct_vehicleowner','gcct_warrantyhistory','gcct_warrantyloanerpartsdelay','gcct_warrantyloanerrequest','gcct_warrantyloanerrequesthistory','globaloptionsetmetadata','incident','incidentresolution','letter','msdyn_answer','msdyn_question','msdyn_questionresponse','msdyn_survey','msdyn_surveyinvite','msdyn_surveyresponse','optionsetmetadata','phonecall','queue','queueitem','sla','slaitem','slakpiinstance','socialactivity','socialprofile','statemetadata','statusmetadata','systemuser','task','team','teammembership','territory']
#for m in sqlContext.tables(database).select('tableName').where("tableName not like '%temptz%'").where("tableName not like '%dup%'").where("tableName not like '%bkp%'").where("tableName not like '%bad%'").collect():
for k in tab_list:
            #k=m.asDict().values()[0].encode('utf-8')
            g1=sqlCtx.sql('desc '+k).toPandas()
            g1['table']=k
            g1=g1[['col_name','data_type','table']]
            df=df.append(g1, ignore_index=True)
            


df.to_excel(writer, sheet_name="sheet1", index=False)
writer.save()
writer.close()

In [3]:
#Schema obtained from MSD
import xlrd
a=pd.ExcelFile('GCCT_EntitySchema_v2.4.xlsx')

In [4]:
e1=sqlContext.tables(database).select('tableName').where("tableName not like '%bkp%'").where("tableName not like '%temptz%'").toPandas()["tableName"]
e2=list(set([i.lower() for sublist in e1 for i in sublist.encode('utf-8').replace("[","").replace("]","").split(", ")]))


# Compare GCCT schema with DSC 

In [9]:
g=set([ h.encode('utf-8').lower() for h in list(a.parse('Account (Account)')['Logical Name'])])

In [10]:
j= set([u.lower() for u in sqlContext.table("account").columns])

In [11]:
len(j.intersection(g))

301

In [99]:
j.difference(g)

{'createdby_entitytype',
 'createdonbehalfby_entitytype',
 'entityid',
 'entitystring',
 'eventdatetime',
 'eventtype',
 'gcct_csmtypecode',
 'gcct_managerid_entitytype',
 'gcct_territorygroup_entitytype',
 'id',
 'lastupdatedatetime',
 'lastupdtby',
 'managerid_entitytype',
 'message__id',
 'modifiedby_entitytype',
 'modifiedonbehalfby_entitytype',
 'organizationid_entitytype',
 'process__id',
 'shakey',
 'sinkcreatedon',
 'sinkmodifiedon',
 'transactioncurrencyid_entitytype'}

In [14]:
len(g.difference(j))

59

In [16]:
len(j),len(g)

(356, 360)

# Comparing GCCT with DSC schema

In [98]:
for i in a.sheet_names:
    b=i.encode('utf-8')
    for c2 in re.findall('\(([^)]+)',b):                
                c=c2.lower()
                #print b,c
                if c in e2:
                    #print("########################################")
                    g=set([ h.encode('utf-8').lower() for h in list(a.parse(b)['Logical Name'])])
                    j= set([u.lower() for u in sqlContext.table(c).columns])
                    print("Table "+str(c)+":"+str(len(j))+" GCCT_Schema:"+str(len(g))+" Common:"+str(len(j.intersection(g)))+" DSC_uniq:"+str(len(j.difference(g)))+" GCCT_uniq:"+str(len(g.difference(j))))
        
    

Table account:356 GCCT_Schema:360 Common:301 DSC_uniq:55 GCCT_uniq:59
Table activityparty:43 GCCT_Schema:32 Common:25 DSC_uniq:18 GCCT_uniq:7
Table annotation:61 GCCT_Schema:41 Common:38 DSC_uniq:23 GCCT_uniq:3
Table appointment:129 GCCT_Schema:116 Common:99 DSC_uniq:30 GCCT_uniq:17
Table businessunit:108 GCCT_Schema:92 Common:86 DSC_uniq:22 GCCT_uniq:6
Table calendar:29 GCCT_Schema:28 Common:11 DSC_uniq:18 GCCT_uniq:17
Table calendarrule:44 GCCT_Schema:46 Common:27 DSC_uniq:17 GCCT_uniq:19
Table contact:430 GCCT_Schema:443 Common:370 DSC_uniq:60 GCCT_uniq:73
Table cxlvhlp_chatactivity:177 GCCT_Schema:159 Common:139 DSC_uniq:38 GCCT_uniq:20
Table cxlvhlp_surveyitem:58 GCCT_Schema:38 Common:35 DSC_uniq:23 GCCT_uniq:3
Table email:160 GCCT_Schema:148 Common:128 DSC_uniq:32 GCCT_uniq:20
Table fax:105 GCCT_Schema:87 Common:78 DSC_uniq:27 GCCT_uniq:9
Table gcct_addportalmessage:131 GCCT_Schema:115 Common:100 DSC_uniq:31 GCCT_uniq:15
Table gcct_caseassignment:134 GCCT_Schema:120 Common:103 DS

# Merging GCCT schema with DSC -- MAPPING DOC

In [26]:
j=sqlContext.sql("desc account").toPandas()
j['col_name']=j['col_name'].str.lower()
g=a.parse("Account (Account)")
g['Schema Name']=g['Schema Name'].str.lower()

In [28]:
pd.merge(g, j, how='right', left_on=['Schema Name'], right_on=['col_name'])

,Logical Name,Schema Name,Display Name,Type,Description,Custom Attribute,Required Level,ValidFor AdvancedFind,Audit Enabled,Secured,Form location,Additional data,col_name,data_type,comment
0,accountcategorycode,accountcategorycode,Account Category,Picklist,Select a category to indicate whether the cust...,False,None,True,True,False,Information/Details/Description,Options:\n1: Preferred Customer\n2: Standard\n...,accountcategorycode,int,None
1,accountclassificationcode,accountclassificationcode,Classification,Picklist,Select a classification code to indicate the p...,False,None,False,True,False,NaN,Options:\n1: Default Value\nDefault: 1,accountclassificationcode,int,None
2,accountid,accountid,Account,Uniqueidentifier,Unique identifier of the account.,False,SystemRequired,True,False,False,NaN,NaN,accountid,string,None
3,accountnumber,accountnumber,Account Number,String,Type an ID number or code for the account to q...,False,None,True,True,False,Information/General/Account Information,Format: Text\nMax length: 20,accountnumber,string,None
4,accountratingcode,accountratingcode,Account Rating,Picklist,Select a rating to indicate the value of the c...,False,None,False,True,False,NaN,Options:\n1: Default Value\nDefault: 1,accountratingcode,int,None
5,address1_addressid,address1_addressid,Address 1: ID,Uniqueidentifier,Unique identifier for address 1.,False,None,False,False,False,NaN,NaN,address1_addressid,string,None
6,address1_addresstypecode,address1_addresstypecode,Address 1: Address Type,Picklist,Select the primary address type.,False,None,True,True,False,Information/General/Address,Options:\n1: Bill To\n2: Ship To\n3: Primary\n...,address1_addresstypecode,int,None
7,address1_city,address1_city,Address 1: City,String,Type the city for the primary address.,False,None,True,True,False,Information/General/Address\nAccount Quick Cre...,Format: Text\nMax length: 80,address1_city,string,None
8,address1_composite,address1_composite,Address 1,Memo,Shows the complete primary address.,False,None,True,True,False,Account for Interactive experience/Summary/ACC...,Format: TextArea\nMax length: 1000,address1_composite,string,None
9,address1_country,address1_country,Address 1: Country/Region,String,Type the country or region for the primary add...,False,None,True,True,False,Information/General/Address,Format: Text\nMax length: 80,address1_country,string,None


In [31]:
#merges dsc gcct data to GCCT descriptions -- ALL DATA
writer = pd.ExcelWriter("GCCT_dsc_desc.xlsx", engine = 'xlsxwriter')

for i in a.sheet_names:
    b=i.encode('utf-8')
    for c2 in re.findall('\(([^)]+)',b):                
                c=c2.lower()
                #print b,c
                if c in e2:
                    #print("########################################")
                    j=sqlContext.sql("desc "+c).toPandas()
                    j['col_name']=j['col_name'].str.lower()
                    j=j[['col_name','data_type']]
                    g=a.parse(b)
                    g['Schema Name']=g['Schema Name'].str.lower()
                    q1=pd.merge(j, g, how='left', right_on=['Schema Name'], left_on=['col_name'])
                    c_name=c[:31]
                    q1.to_excel(writer, sheet_name=c_name, index=False)

writer.save()
writer.close()
        
    

In [33]:
#merges dsc gcct data to GCCT descriptions -- LIMITED COLUMNS
writer = pd.ExcelWriter("GCCT_dsc_desc.xlsx", engine = 'xlsxwriter')

for i in a.sheet_names:
    b=i.encode('utf-8')
    for c2 in re.findall('\(([^)]+)',b):                
                c=c2.lower()
                #print b,c
                if c in e2:
                    #print("########################################")
                    j=sqlContext.sql("desc "+c).toPandas()
                    j['col_name']=j['col_name'].str.lower()
                    j=j[['col_name','data_type']]
                    g=a.parse(b)
                    g['Schema Name']=g['Schema Name'].str.lower()
                    g=g[['Schema Name','Display Name','Description','Additional data']]
                    q1=pd.merge(j, g, how='left', right_on=['Schema Name'], left_on=['col_name'])
                    q1=q1[['col_name','data_type','Display Name','Description','Additional data']]
                    c_name=c[:31]
                    q1.to_excel(writer, sheet_name=c_name, index=False)

writer.save()
writer.close()
        
    